Stage 1- splitting the data based on the on/off times of each alarm instance

Change directories, specify the necessary new folder locations and get the list of csv files for the turbine data. Files are opened and read manually instead of loading in since they are too large for memory

In [1]:
import pandas as pd
import numpy as np
import glob,os


dir = "D:/WindData/POD_Request_3541/Project"

os.chdir(dir)
if not os.path.exists('processed'):
    os.makedirs('processed')

names = glob.glob("Copies/*.csv")
print(names)

['Copies\\LDT-1Hz-01-January.csv', 'Copies\\LDT-1Hz-02-February.csv', 'Copies\\LDT-1Hz-03-March.csv', 'Copies\\LDT-1Hz-04-April.csv', 'Copies\\LDT-1Hz-05-May.csv', 'Copies\\LDT-1Hz-06-June.csv', 'Copies\\LDT-1Hz-07-July.csv', 'Copies\\LDT-1Hz-08.csv', 'Copies\\LDT-1Hz-09-September.csv', 'Copies\\LDT-1Hz-Tur-2017-10.csv', 'Copies\\LDT-1Hz-Tur-2017-11.csv', 'Copies\\LDT-1Hz-Tur-2017-12.csv', 'Copies\\LDT-1Hz-Tur-2018-01.csv', 'Copies\\LDT-1Hz-Tur-2018-02.csv', 'Copies\\LDT-1Hz-Tur-2018-03.csv', 'Copies\\LDT-1Hz-Tur-2018-04.csv', 'Copies\\LDT-1Hz-Tur-2018-05.csv', 'Copies\\LDT-1Hz-Tur-2018-06.csv', 'Copies\\LDT-1Hz-Tur-2018-07.csv', 'Copies\\LDT-1Hz-Tur-2018-08.csv', 'Copies\\LDT-1Hz-Tur-2018-09.csv', 'Copies\\LDT-1Hz-Tur-2018-10.csv', 'Copies\\LDT-1Hz-Tur-2018-11.csv', 'Copies\\LDT-1Hz-Tur-2018-12.csv', 'Copies\\LDT-1Hz-Tur-2019-01.csv', 'Copies\\LDT-1Hz-Tur-2019-02.csv', 'Copies\\LDT-1Hz-Tur-2019-03.csv', 'Copies\\LDT-1Hz-Tur-2019-05.csv', 'Copies\\LDT-1Hz-Tur-2019-06.csv', 'Copies\\LDT

Load the list of alarms you want to seperate your data based on. Then create a list of on and off times for the alarms

In [2]:
alarms = pd.read_csv("Alarms.csv")

In [3]:
ons = alarms['On'].to_list()
offs = alarms['Off'].to_list()

In [4]:
print(ons)
print(offs)

['18/02/2017 07:12', '08/03/2017 16:30', '17/03/2017 18:51', '24/03/2017 09:10', '24/03/2017 22:30', '16/11/2017 17:43', '21/05/2018 06:16', '22/05/2018 06:17', '23/05/2018 06:16', '24/05/2018 06:21', '25/05/2018 06:06', '28/05/2018 06:07', '29/05/2018 06:12', '30/05/2018 06:24', '31/05/2018 06:16', '01/06/2018 06:18', '04/06/2018 06:22']
['23/02/2017 16:08', '17/03/2017 15:42', '23/03/2017 15:30', '24/03/2017 15:20', '25/03/2017 00:45', '18/12/2017 00:00', '21/05/2018 18:04', '22/05/2018 18:13', '23/05/2018 18:12', '24/05/2018 18:11', '25/05/2018 14:08', '28/05/2018 18:16', '29/05/2018 17:48', '30/05/2018 18:30', '31/05/2018 17:26', '01/06/2018 14:05', '04/06/2018 17:08']


In [5]:
name = "Copies\\LDT-1Hz-01-January.csv"
file = open(name,'r')
columns = file.readline()
file.close()

Specify the functions needed to iterate through the datasets. 

In [6]:


fileList = []
currentFile = 0

#Read a new line of data, if the file is empty load a new file and read again. Retrun the line of data and the current file
def iterateLine(file):
    global currentFile
    line = file.readline()
    if(line==""):
        file.close()
        currentFile+=1
        file = open(fileList[currentFile], 'r')
        file.readline()
        line = file.readline()
    return file, line
#Read a new line of data, if the file is empty load a new file and read again. Discard the line of data and return the current file
def deleteLine(file):
    global currentFile
    line = file.readline()
    if(line==""):
        file.close()
        currentFile+=1
        file = open(fileList[currentFile], 'r')
        file.readline()
        file.readline()
    return file

#Convert the dataline time and date format so it matches the alarm time and date format
def getExactTime(line):
    exact = line.split(",")[0]
    split = exact.split()
    date, time = split[0], split[1]
    dateSplit = date.split("-")
    dateFix = dateSplit[2] + "/" + dateSplit[1] + "/" + dateSplit[0]
    return dateFix + " " + time


# Check if the date and time for the data matches the date and time for the alarm
def timeMatches(dataLine, alarmLine):
    fixedDataLine = getExactTime(dataLine)
    if alarmLine in fixedDataLine:
        return False
    else:
        return True


In [7]:
alarmLine = "18/02/2017 07:12"
line = "2017-02-17 07:12:00.608"
print(timeMatches(line,alarmLine))

True


Keep track of the line counts of each new file generated

In [8]:
lineCounts = []

Core split by alarm loop. The function is given a list of places it can read and write to, as well as the alarm on and off times it is searching for. It splits the data it can read from based on the alarm on/off times and puts the data in the write locations. There is one file generated for every alarm occurrence. 

In [9]:
def createMySplits(writePaths, ons, offs, dataLine, readFile):
    for (writePath, on, off) in zip(writePaths, ons, offs):
        writeFile = open(writePath, 'w')
        writeFile.write(columns)
        lineCount = 0
        while timeMatches(dataLine, on):
            writeFile.write(dataLine)
            readFile,dataLine = iterateLine(readFile)
            lineCount+=1
        print("First round done")
        while timeMatches(dataLine, off):
            readFile, dataLine = iterateLine(readFile)
        print("Found All offlines")
        while not timeMatches(dataLine, off):
            readFile,dataLine = iterateLine(readFile)
        print("Resetting")
        writeFile.close()
        lineCounts.append(lineCount)

Generate the available writeto files based on how many alarm instances there are

In [10]:
writeDir = "./processed/"
writePaths = []
for i in range(len(ons)):
    pathVal = writeDir + str(i) + ".csv"
    writePaths.append(pathVal)


Customise createMySplits search. This is done so that excess data is never searched- e.g. sometimes the alarm might not occur for months, causing the function to search through months of second-to-second data, and all those months of data will end up in one file as it is the data leading up to one alarm instance. The createMySplits function can still be run on the whole dataset at once instead of customising it, it just takes much longer.

In [11]:
onGroups = [ons[0:5], ons[5:6], ons[6:]]
offGroups = [offs[0:5], offs[5:6], offs[6:]]
writePathGroups = [writePaths[0:5], writePaths[5:6], writePaths[6:]]
readPathGroups = [names[1:3], names[10:12], names[16:18]]
#print(readPathGroups)

In [12]:
#for (onGroup, offGroup, writePathGroup, readPathGroup) in zip(onGroups, offGroups, writePathGroups, readPathGroups):
#    print(onGroup)
#    print(offGroup)
#    print(writePathGroup)
#    print(readPathGroup)
#    print()

In [13]:
for (onGroup, offGroup, writePathGroup, readPathGroup) in zip(onGroups, offGroups, writePathGroups, readPathGroups):
    currentFile = 0
    fileList = readPathGroup
    openFile = open(fileList[0], 'r')
    openFile.readline()
    openFile.readline()
    dataLine = openFile.readline()
    createMySplits(writePathGroup, onGroup, offGroup, dataLine, openFile)
    
print("Done")

First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
First round done
Found All offlines
Resetting
Done
